In [ ]:
pip install transformers datasets evaluate

In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.7 MB/s eta 0:00:00


In [ ]:
# ML libraries
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split

# Pytorch ML libraries
import transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSeq2SeqLM, T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorWithPadding
import torch
from torch.utils.data import Dataset, DataLoader

# Misc.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
if torch.cuda.is_available():
    torch.device('cuda')
    print('cuda')
else:
    torch.device('cpu')

cuda


In [ ]:
from google.colab import files
uploaded = files.upload()

train = open('train.csv')
train_file = pd.read_csv(train)

dev = open('validation.csv')
dev_file = pd.read_csv(dev)

test = open('test.csv')
test_file = pd.read_csv(test)

Saving test.csv to test.csv
Saving train.csv to train.csv
Saving validation.csv to validation.csv


In [ ]:
gen_train_file = [train_file.iloc[i] for i in range(train_file.shape[0]) if train_file.iloc[i]['category'] == 'general' ]
gen_dev_file = [dev_file.iloc[i] for i in range(dev_file.shape[0]) if dev_file.iloc[i]['category'] == 'general' ]
gen_test_file = [test_file.iloc[i] for i in range(test_file.shape[0]) if test_file.iloc[i]['category'] == 'general' ]

In [ ]:
gen_train_df = pd.DataFrame(gen_train_file)
gen_train_df = gen_train_df.reset_index()

gen_dev_df = pd.DataFrame(gen_dev_file)
gen_dev_df = gen_dev_df.reset_index()

gen_test_df = pd.DataFrame(gen_test_file)
gen_test_df = gen_test_df.reset_index()

want to seperate question from context..

some problems only have the question so i have to repeat the problem for question and context, or have no context

In [ ]:
gen_test_df.iloc[9].problem

'a profit of rs . 900 is divided between x and y in the ratio of 1 / 2 : 1 / 3 . what is the difference between their profit shares ?'

In [ ]:
def flatten_tree(node):
    if 'children' in node:
        left_expr = flatten_tree(node['children'][0])
        operator = node['name']
        right_expr = flatten_tree(node['children'][1])
        return f"{operator}({left_expr}, {right_expr})"
    else:
        return node['name']

Ops = {'-': 'subtract',
        '+': 'add',
        '/': 'divide',
        '*':'multiply'}


tree = {'name': '-', 'children': [{'name': '/', 'children': [{'name': '/', 'children': [{'name': '900'}, {'name': '+', 'children': [{'name': '/', 'children': [{'name': '1'}, {'name': '2'}]}, {'name': '/', 'children': [{'name': '1'}, {'name': '3'}]}]}]}, {'name': '2'}]}, {'name': '/', 'children': [{'name': '/', 'children': [{'name': '900'}, {'name': '+', 'children': [{'name': '/', 'children': [{'name': '1'}, {'name': '2'}]}, {'name': '/', 'children': [{'name': '1'}, {'name': '3'}]}]}]}, {'name': '3'}]}]}

flattened_tree = flatten_tree(tree)


print(flattened_tree)


-(/(/(900, +(/(1, 2), /(1, 3))), 2), /(/(900, +(/(1, 2), /(1, 3))), 3))


In [ ]:
import re

def convert_expression(input_expression):
    # Replace characters in the input expression to match the desired syntax
    pattern = input_expression.replace('/', 'divide').replace('+', 'add').replace('-', 'subtract').replace('*', 'multiply')
    return pattern

# Example usage
input_expression = '-(/(/(900, +(/(1, 2), /(1, 3))), 2), /(/(900, +(/(1, 2), /(1, 3))), 3))'
converted_expression = convert_expression(flattened_tree)
print(converted_expression)


subtract(divide(divide(900, add(divide(1, 2), divide(1, 3))), 2), divide(divide(900, add(divide(1, 2), divide(1, 3))), 3))


# Tokenize

tokenize for the question + context

In [ ]:
gen_train_df.shape[0] - 6000

1231

In [ ]:
structured_train_df = pd.DataFrame()

for i in range(gen_train_df.shape[0] - 6000):

  problem = gen_train_df.iloc[i].problem
  problem = problem.strip()
  problem = problem.split('.')

  # problem = gen_train_df['problem'][i]
  question = problem[-1]
  context = np.ravel(problem[:-1])
  annotated_formula = gen_train_df.iloc[i].annotated_formula
  formula = gen_train_df.iloc[i].formula
  exp_tree = gen_train_df.iloc[i].tree

  context_sentence = (f'{question} </sep> {context}')

  # context_sentence = (f'''This is an example of an annotated formula: divide(72, multiply(6, 210)).
  #                     What is the annotated formula to this question: {question if 'what' or '?' in question else context}
  #                     given the following context: {context if 'what' or '?' not in context else question}

  #                     Let's think step-by-step.''')
  row_to_add = pd.DataFrame([{'question':context_sentence, 'labels':annotated_formula}])
  structured_train_df = pd.concat([structured_train_df, row_to_add])


structured_train_df = structured_train_df.reset_index()
structured_train_df = structured_train_df.drop(columns=["index"])
structured_train_df.head()

,question,labels
0,how long is her book ? </sep> ['sophia finish...,"divide(90, subtract(const_1, divide(2, 3)))"
1,</sep> ['a student was asked to find 4 / 5 of...,"divide(divide(multiply(multiply(36, divide(4, ..."
2,what is the new concentration of mixture ? </...,"multiply(divide(add(multiply(divide(30, const_..."
3,the no of 20 paise coins is </sep> ['the tota...,"divide(subtract(multiply(324, 25), multiply(70..."
4,"what was the cost , in dollars , of the large...","add(divide(subtract(8.25, multiply(divide(mult..."


In [ ]:
(gen_dev_df.shape[0]) - ((gen_dev_df.shape[0]) * 0.17)

896.4

In [ ]:
structured_val_df = pd.DataFrame()

for i in range(gen_dev_df.shape[0] - 896):

  problem = gen_dev_df.iloc[i].problem
  problem = problem.strip()
  problem = problem.split('.')

  # problem = gen_train_df['problem'][i]
  question = problem[-1]
  context = np.ravel(problem[:-1])
  annotated_formula = gen_dev_df.iloc[i].annotated_formula

  context_sentence = (f'{question} </sep> {context}')
  row_to_add = pd.DataFrame([{'question':context_sentence, 'labels':annotated_formula}])
  structured_val_df = pd.concat([structured_val_df, row_to_add])


structured_val_df = structured_val_df.reset_index()
structured_val_df = structured_val_df.drop(columns=["index"])
structured_val_df.head()

,question,labels
0,in how many r ways can the test be completed ...,"power(5, 4)"
1,what is the probability that the product of i...,"subtract(const_1, power(divide(const_1, const_..."
2,"if m + n = 72 , then 1 / m + 1 / n is equal t...","divide(72, multiply(6, 210))"
3,if a person is selected at random from this g...,"divide(subtract(160, 90), 160)"
4,if 1 / 3 of the pieces not on display are pai...,"divide(divide(800, subtract(1, divide(1, 3))),..."


In [ ]:
(gen_test_df.shape[0]) - ((gen_test_df.shape[0]) * 0.17)

595.11

In [ ]:
structured_test_df = pd.DataFrame()

for i in range(gen_test_df.shape[0] - 595):

  problem = gen_test_df.iloc[i].problem
  problem = problem.strip()
  problem = problem.split('.')

  # problem = gen_train_df['problem'][i]
  question = problem[-1]
  context = np.ravel(problem[:-1])
  annotated_formula = gen_test_df['annotated_formula'][i]

  context_sentence = (f'{question} </sep> {context}')

  row_to_add = pd.DataFrame([{'question':context_sentence, 'labels':annotated_formula}])
  structured_test_df = pd.concat([structured_test_df, row_to_add])

structured_test_df = structured_test_df.reset_index()
structured_test_df = structured_test_df.drop(columns=["index"])
structured_test_df.head()

,question,labels
0,the sum of all the integers s such that - 26 <...,"add(add(negate(26), const_1), add(add(negate(2..."
1,how many zeroes will he need ? </sep> ['there...,"add(add(divide(1000, const_10), multiply(subtr..."
2,how many statues can she paint ? </sep> ['dia...,"divide(divide(1, 2), divide(1, 16))"
3,if the price of gasoline increases by 25 % and...,"multiply(divide(subtract(add(25, const_100), a..."
4,if 1 / 3 of the pieces not on display are pai...,"divide(divide(1000, subtract(const_1, divide(1..."


In [ ]:
structured_train_df.to_csv("train_data.csv", index=False)
structured_val_df.to_csv("dev_data.csv", index=False)
structured_test_df.to_csv("test_data.csv", index=False)

from datasets import load_dataset

dataset = load_dataset('csv',
                        data_files={'train': 'train_data.csv', 'dev': 'dev_data.csv', 'test': 'test_data.csv'})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
pip install sentencepiece

In [ ]:
def get_nums_from_problem(data, convert_to_float=False):
    nums = []
    problems = []
    for problem in data['question']:
        num = re.compile('([+-]?((\d+(\.\d*)?)|(\.\d+)))')
        big = re.compile(r'(-?\d{1,3}(,\d{3})+(\.\d*)?)')

        big_results = re.finditer(big, problem)
        problem = re.sub(big, NUM_MASK, problem)
        num_results = re.finditer(num, problem)
        problem = re.sub(num, NUM_MASK, problem)

        # Getting the combined numbers in order of occurence
        combined = [x for x in num_results]
        combined.extend([x for x in big_results])
        combined = sorted(combined, key=lambda x: x.start(0))

        if convert_to_float:
            combined = [float(x.group(0).replace(',','')) for x in combined]
        else:
            combined = [x.group(0) for x in combined]

        nums.append(combined)
        problems.append(problem)
    return nums, problems

In [ ]:
device = torch.device('cuda')
cpu_device = torch.device('cpu')

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import sentencepiece
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
model = model.to(device)

NUM_MASK = '<num>'
tokenizer.add_special_tokens({'additional_special_tokens':[NUM_MASK]})
model.resize_token_embeddings(len(tokenizer)).to(device)

def preprocess_function(examples):
  # tokenization = tokenizer(examples['question'], examples['answer'], padding='max_length', max_length=329, truncation=True)

  _,problem = get_nums_from_problem(examples)

  def token(x):
    return tokenizer(x, padding='max_length', max_length=150, truncation=True)

  tokenized = list(map(token, problem))
  tokenized_labels = list(map(token, examples['labels']))

  input_ids = torch.stack([torch.tensor(x['input_ids']) for x in tokenized]).to(device)
  attention_mask = torch.stack([torch.tensor(x['attention_mask']) for x in tokenized]).to(device)
  labels = torch.stack([torch.tensor(x['input_ids']) for x in tokenized_labels]).to(device)

  examples["input_ids"] = input_ids
  examples["attention_mask"] = attention_mask
  examples["labels"] = labels


  # return {'input_ids':input_ids.long(), 'attention_mask':attention_mask.int()}

  # inputs = tokenizer(examples["answer"], examples["question"], padding='max_length', max_length=392, truncation=True, return_tensors="pt").to(device)
  # output_sequences = model.generate(input_ids=input_ids, attention_mask=attention_mask).to(device)
  # print(output_sequences)
  # sequences = tokenizer.batch_decode(output_sequences)
  return examples

dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1231 [00:00<?, ? examples/s]

Map:   0%|          | 0/184 [00:00<?, ? examples/s]

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

# Tutorial

In [ ]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['dev'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [ ]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['labels', 'input_ids', 'attention_mask'])


An example

In [ ]:
tokenizer.decode(batch['input_ids'][0])

'the smallest value of n, for which <num> n + <num> is not a prime number, is <unk>/sep> []</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [ ]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'add(2, 2)</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [ ]:
pip install rouge_score

In [ ]:
import evaluate
accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    predictions, references = eval_pred
    print(eval_pred)
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=references)

In [ ]:
# from transformers import DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
train_dataset = dataset['train']
eval_dataset = dataset['dev']

In [ ]:
# flat_train_dataset = train_dataset.flatten()

In [ ]:
# updated_train_dataset = train_dataset.map(lambda example: {"input_ids": example["input_ids"][0]}, remove_columns=["input_ids"])
# updated_train_dataset = updated_train_dataset.map(lambda example: {"attention_mask": example["attention_mask"][0]}, remove_columns=["attention_mask"])

# updated_eval_dataset = eval_dataset.map(lambda example: {"input_ids": example["input_ids"][0]}, remove_columns=["input_ids"])
# updated_eval_dataset = updated_eval_dataset.map(lambda example: {"attention_mask": example["attention_mask"][0]}, remove_columns=["attention_mask"])

In [ ]:
train_dataset['input_ids']

tensor([[149, 307,  19,  ...,   0,   0,   0],
        [  3,   2,  87,  ...,   0,   0,   0],
        [125,  19,   8,  ...,   0,   0,   0],
        ...,
        [227,  48, 433,  ...,   0,   0,   0],
        [  3,  99,   3,  ...,   0,   0,   0],
        [125,  19,   8,  ...,   0,   0,   0]])

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
import nltk

# Setup evaluation
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = str(labels)[1:-1]
    print(labels)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    print(decoded_labels)


    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

# Load pretrained model and evaluate model after each epoch
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


In [ ]:
from transformers import Trainer
from transformers import TrainingArguments

batch_size = 2
model_name = "t5-base-medium-title-generation"
model_dir = f"drive/MyDrive/Models/{model_name}"

training_args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=50,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="steps",
    save_steps=100,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss


[  579   599 11116 ...     0     0     0]
 [31836   599  8056 ...     0     0     0]
 [14514   599  5865 ...     0     0     0]
 ...
 [14514   599 23829 ...     0     0     0]
 [14514   599  2128 ...     0     0     0]
 [  617   599  2079 ...     0     0     0]


TypeError: ignored